In [2]:
import pandas_datareader as web
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import datetime
from dateutil.relativedelta import relativedelta
from scipy.optimize import LinearConstraint
import plotly.express as px
import plotly.graph_objects as go
from dateutil import rrule
from datetime import date
import os
from plotly.subplots import make_subplots

In [3]:
class PreProcess:
    
    def __init__(self, start_date,end_date): #파라미터 : 사용자가 입력한 날짜
        self.start_date = datetime.datetime.strptime(start_date, '%Y%m')
        self.end_date = datetime.datetime.strptime(end_date, '%Y%m')
        
    def calculate_period(self,start_date, end_date): #기간 계산
        return (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)  

    def return_dataframe(self,start_date,end_date):
        #start_date = start_date-relativedelta(years=1) #lookback 기간
        data1 = pd.read_csv('Assets_1.csv')
        data1.index = pd.to_datetime(data1['Unnamed: 0'])
        del data1['Unnamed: 0']
        data1.index.names=['Year']
        
        #data2 = fama 10 industry
        data2 = pd.read_csv('Assets_2.csv')
        data2.index = pd.to_datetime(data2['Unnamed: 0'])
        del data2['Unnamed: 0']
        data2.index.names=['Year']
        return data1, data2

    def return_lookback(self,data, start_date,lookback):
        lookback_start = start_date-relativedelta(years=int(lookback))
        lookback_end = start_date-relativedelta(months=1)
        data_lookback = data.loc[lookback_start:lookback_end]
        return data_lookback
    
class predict:
    def __init__(self):
        pass
    
    def get_data_econ():
        """
        raw한 x값 
        """
        econ = pd.read_csv('new_econ.csv')
        econ = econ.set_index('DATE')
        econ.index = econ.index.astype('datetime64[ns]') 
        return econ
    
    def get_data_sp():
        """
        1960-1999 s&p 데이터 수익률
        """
        sp = pd.read_csv('SP_ret.csv')
        sp = sp.set_index('Date')
        sp.index = sp.index.astype('datetime64[ns]') 
        return sp 
    
    def get_data_sp1():
        """
        +/- 로 만 나눈거 (positive,negative)
        """
        sp1 = pd.read_csv('sp1_F.csv')
        sp1 = sp1.set_index('Date')
        sp1.index = sp1.index.astype('datetime64[ns]') 
        return sp1
    def get_data_sp2():
        """
        Moderate / Strong 나눈거 
        """
        sp2 = pd.read_csv('sp2_REAL_F.csv ')
        sp2 = sp2.set_index('Date')
        sp2.index = sp2.index.astype('datetime64[ns]') 
        return sp2
    
    def get_data_econ_extra_test_():
        """
        2000년 이후 X 값 
        """
        econt = pd.read_csv('z_test.csv')
        econt = econt.set_index('DATE')
        econt.index = econt.index.astype('datetime64[ns]') 
        return econt
    
    def get_data_test():
        """
        test dataframe 2000 ~ 
        """
        
        test = pd.read_csv('test_pre.csv')
        test = test.set_index('DATE')
        test.index = test.index.astype('datetime64[ns]') 
        return test
    
    def get_data_sp1t():

        sp1t = pd.read_csv('sp1t.csv')
        sp1t  = sp1t.set_index('Date')
        sp1t.index = sp1t.index.astype('datetime64[ns]') 
        return sp1t
   
    def get_data_sp2t():

        sp2t = pd.read_csv('sp2t.csv')
        sp2t  = sp1t.set_index('Date')
        sp2t.index = sp2t.index.astype('datetime64[ns]') 
        return sp2t
    
    def trans_to_X1(X):
        """
        X1 로 변형
        """
        return np.array([X.iloc[:,:-1]])
    
    def trans_to_y1(y):
        """
        y1 로 변형
        """
        return np.array([y.iloc[:,-1]])
         
    def split(X1,y1,tsize):
        from sklearn.model_selection import train_test_split
        return train_test_split(X1, y1, test_size=size, random_state=321)
    
    def result(start,end):
        econ = predict.get_data_econ()
    def get_model1(start,end):
        """
        모델 1 
        """
        # 2000년이후 데이타 갖고오기 
        test = predict.get_data_econ_1_test()
        test=test.iloc[:,:-1]
        test.dropna(inplace=True)
        test = test.iloc[:,:]
        
        list_time = test.index
        test_start=0
        test_end=0
        
        # 시간 조정하기 
        for x in range(len(list_time)):
            if list_time[x]<=start<=list_time[x+1]:
                test_start = x+1
        for x in range(len(list_time)):
            if list_time[x]<=end<=list_time[x+1]:
                test_end = x+1
        test = test.iloc[test_start:test_end+1,:]

        import pickle
        filename = 'final_model1.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
        pre = loaded_model.predict(np.array(test))
        
        # 결과값 
        res=pd.DataFrame()
        res['Date'] = test.index
        res['Prediction'] = pre
        res=res.set_index('Date')
        return res

    
    def get_model2(start,end):
        """
        모델2 (Pos Mos 구분)
        """
        # 2000년이후 데이타 갖고오기 
        test = predict.get_data_econ_1_test()
        test=test.iloc[:,:-1]
        test.dropna(inplace=True)
        test = test.iloc[:,:]
        
        list_time = test.index
        test_start=0
        test_end=0
        
        # 시간 조정하기 
        for x in range(len(list_time)):
            if list_time[x]<=start<=list_time[x+1]:
                test_start = x+1
                
        for x in range(len(list_time)):
            if list_time[x]<=end<=list_time[x+1]:
                test_end = x+1
        
        test = test.iloc[test_start:test_end+1,:]

        import pickle
        filename = 'final_model2.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
        pre = loaded_model.predict(np.array(test))
        
          # 결과값 
        res=pd.DataFrame()
        res['Date'] = test.index
        res['Prediction'] = pre
        res=res.set_index('Date')
        return res
    
    def get_data_econ_1_train():

        sp1 = pd.read_csv('econ_1_train.csv')
        sp1 = sp1.set_index('DATE')
        sp1.index = sp1.index.astype('datetime64[ns]') 
        return sp1
    
    def get_data_econ_2_train():

        sp1 = pd.read_csv('econ_2_train.csv')
        sp1 = sp1.set_index('DATE')
        sp1.index = sp1.index.astype('datetime64[ns]') 
        return sp1
    
    def get_data_econ_1_test():

        sp1 = pd.read_csv('econ_1_test.csv')
        sp1 = sp1.set_index('DATE')
        sp1.index = sp1.index.astype('datetime64[ns]') 
        return sp1
    
    def get_data_econ_2_test():

        sp1 = pd.read_csv('econ_2_test.csv')
        sp1 = sp1.set_index('DATE')
        sp1.index = sp1.index.astype('datetime64[ns]') 
        return sp1
    
    def eco_status_predict(start_date, end_date):
        df1 = predict.get_model1(start_date, end_date)  # start, end date 입력 
        df2 = predict.get_model2(start_date, end_date)

        df1.rename(columns={'Prediction':'model1_pre'}, inplace=True)
        df2.rename(columns={'Prediction':'model2_pre'}, inplace=True)

        df = pd.concat([df1,df2],axis=1)

        # 조건 선택
        conditions = [
            (df['model1_pre'] == 1), #pos
            (df['model1_pre'] == 0) & (df['model2_pre'] == 2), #mod
            (df['model1_pre'] == 0) & (df['model2_pre'] == 3)  #neg
            ]

        # 값 선택
        values = [1,2,3]

        # 파생변수 생성
        df['final'] = np.select(conditions, values)
        df_final = df['final']
        df_final_arr = np.array(df_final)

        return df_final_arr
    
class optimize_portfolio:
    def __init__(self, dataset, asset_type):
        if asset_type == '1':
            self.asset_num = 5
            self.asset_type = asset_type

        elif asset_type == '2':
            self.asset_num = 10      
            self.asset_type = asset_type
        self.data = dataset
        self.mean_returns = self.data.mean()  #월별 mean_return
        self.cov_matrix = self.data.cov()  #월별 공분산행렬
        
    def min_variance(self,mean_returns, cov_matrix,inv_style,eco_status,pre_weight):
        self.num_assets = self.asset_num
        rate=0.1
        if self.asset_type =='1':
            fun =lambda x: np.dot(np.dot(np.array([x[0],x[1],x[2],x[3],x[4]]).T,cov_matrix),np.array([x[0],x[1],x[2],x[3],x[4]]))
            constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
            constraints.append({'type':'ineq','fun':lambda x: np.dot(x,1)-0})
            if inv_style ==1:#안정형:
                #Cnsmr[0], Manuf[1], Hitec[2], Hlth[3]
                if eco_status==3: #pos
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4]]),pre_weight)).sum()})  
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[0]-x[2]})   #pos 일때는 Cnsmr -> Hitec -> Hlth->Manuf
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[0]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.6-x[4]})
                elif eco_status==2: #mod
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4]]),pre_weight)).sum()})  
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[1]-x[0]})   #mod 일때는 Manuf ->Hlth->Cnsmr->HiTec
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[1]-x[2]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[2]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.6-x[4]})
                else: #neg
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4]]),pre_weight)).sum()})  
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[1]})   #neg 일때는 HiTec->Hlth->Manuf->Cnsmr
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[0]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[0]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.6-x[4]})
            elif inv_style==2:
                if eco_status==3: #pos
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4]]),pre_weight)).sum()})  
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[0]-x[2]})   #pos 일때는 Cnsmr -> Hitec -> Hlth->Manuf
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[0]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.4-x[4]})
                elif eco_status==2: #mod
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4]]),pre_weight)).sum()})  
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[1]-x[0]})   #mod 일때는 Manuf ->Hlth->Cnsmr->HiTec
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[1]-x[2]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[2]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.4-x[4]})
                else: #neg
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4]]),pre_weight)).sum()})  
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[1]})   #neg 일때는 HiTec->Hlth->Manuf->Cnsmr
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[0]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[0]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.4-x[4]})
            else:
                if eco_status==3: #pos
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4]]),pre_weight)).sum()})  
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[0]-x[2]})   #pos 일때는 Cnsmr -> Hitec -> Hlth->Manuf
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[0]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.2-x[4]})
                elif eco_status==2: #mod
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4]]),pre_weight)).sum()})  
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[1]-x[0]})   #mod 일때는 Manuf ->Hlth->Cnsmr->HiTec
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[1]-x[2]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[2]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.2-x[4]})
                else: #neg
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4]]),pre_weight)).sum()})  
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[1]})   #neg 일때는 HiTec->Hlth->Manuf->Cnsmr
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[0]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[0]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.2-x[4]})
                    
        if self.asset_type =='2':
            #NoDur[0],Durbl[1],Manuf[2],Energy[3],Hitec[4],Telcm[5],Shops[6], Hlth[7], Utils[8]
            fun =lambda x: np.dot(np.dot(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]).T,cov_matrix),
                                  np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]))
            constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
            constraints.append({'type':'ineq','fun':lambda x: np.dot(x,1)-0})
            if inv_style ==1:#안정형:
                if eco_status==3: #pos
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]),
                                                                                             pre_weight)).sum()})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[2]})   #pos 일때는 Shops->HiTec->Nodur->Hlth->Durbl->Manuf->Telcm->Energy->Utils
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[5]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[3]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[5]})
                    
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[4]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[0]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[7]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.6-x[9]})
                elif eco_status==2: #mod
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]),
                                                                                             pre_weight)).sum()})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[0]})   #mod일때 Energy->Durbl->Manuf->Hlth->Nodur->Shops->Utils->Telcm->HiTec
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[6]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[5]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[4]})
                    
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[1]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[7]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.6-x[9]})
                else: #neg
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]),
                                                                                             pre_weight)).sum()})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[0]})   #neg일때 Telcm->Energy->Utils->Hlth->NoDur->HiTec->Manuf->Shops->Durbl
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[2]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[6]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[1]})
                    
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[8]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[7]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.6-x[9]})
            elif inv_style==2:
                if eco_status==3: #pos
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]),
                                                                                             pre_weight)).sum()})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[2]})   #pos 일때는 Shops->HiTec->Nodur->Hlth->Durbl->Manuf->Telcm->Energy->Utils
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[5]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[3]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[5]})
                    
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[4]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[0]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[7]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.4-x[9]})
                elif eco_status==2: #mod
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]),
                                                                                             pre_weight)).sum()})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[0]})   #mod일때 Energy->Durbl->Manuf->Hlth->Nodur->Shops->Utils->Telcm->HiTec
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[6]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[5]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[4]})
                    
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[1]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[7]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.4-x[9]})
                else: #neg
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]),
                                                                                             pre_weight)).sum()})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[0]})   #neg일때 Telcm->Energy->Utils->Hlth->NoDur->HiTec->Manuf->Shops->Durbl
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[2]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[6]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[1]})
                    
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[8]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[7]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.4-x[9]})
            else:
                if eco_status==3: #pos
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]),
                                                                                             pre_weight)).sum()})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[2]})   #pos 일때는 Shops->HiTec->Nodur->Hlth->Durbl->Manuf->Telcm->Energy->Utils
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[5]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[3]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[6]-x[5]})
                    
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[4]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[0]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[7]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.2-x[9]})
                elif eco_status==2: #mod
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]),
                                                                                             pre_weight)).sum()})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[0]})   #mod일때 Energy->Durbl->Manuf->Hlth->Nodur->Shops->Utils->Telcm->HiTec
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[6]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[8]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[5]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[4]})
                    
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[1]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[2]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[7]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.2-x[9]})
                else: #neg
                    constraints.append({'type': 'ineq', 'fun': lambda x: rate-abs(np.subtract(np.array([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]]),
                                                                                             pre_weight)).sum()})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[0]})   #neg일때 Telcm->Energy->Utils->Hlth->NoDur->HiTec->Manuf->Shops->Durbl
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[4]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[2]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[6]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[5]-x[1]})
                    
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[3]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[8]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: x[7]-x[1]})
                    constraints.append({'type': 'ineq', 'fun': lambda x: 0.2-x[9]})
            
            
        bound = (0.0,1.0)
        bounds = tuple(bound for asset in range(self.num_assets))
        for turnover_rate in list(np.linspace(0.1,1,10)):
            rate = turnover_rate
            result =minimize(fun, self.num_assets*[1./self.num_assets], method='SLSQP', bounds=bounds,constraints=constraints)
            #print(result['success'], rate)
            if result['success'] == True:
                break
        return result

class Backtest:
    def __init__(self, start_date, end_date, money, asset_type):
        self.df = PreProcess(start_date, end_date)
        self.start_date = self.df.start_date
        self.end_date = self.df.end_date
        self.df_investment1, self.df_investment2 = self.df.return_dataframe(self.start_date, self.end_date)
        self.df_investment1 = self.df_investment1*100
        self.df_investment2 = self.df_investment2*100
        self.asset_type=asset_type
        if asset_type =='1':
            self.df_investment = self.df_investment1.copy()
            self.asset_type=asset_type
        elif asset_type == '2':
            self.df_investment = self.df_investment2.copy()
        
        #df_investment1 = 5indsutry, df_investment2 = 10industry
        self.start_date_list = list(rrule.rrule(rrule.MONTHLY, dtstart=self.df.start_date, until=self.df.end_date))
        self.money = money
        self.asset_num = len(self.df_investment.columns)
        
    def get_rebalance_weight_optimize_portfolio(self, start_date_list,inv_style,eco_status_arr,lookback):
        weight_arr = np.array([])
        if self.asset_type=='1':
            if inv_style==1:
                pre_weight= [0.1,0.1,0.1,0.1,0.6]
            elif inv_style==2:
                pre_weight= [0.15,0.15,0.15,0.15,0.4]
            else:
                pre_weight= [0.2,0.2,0.2,0.2,0.2]
        elif self.asset_type=='2':
            if inv_style==1:
                pre_weight= [0.044,0.044,0.044,0.044,0.044,0.044,0.044,0.044,0.044,0.6] #초기 Weight
            elif inv_style==2:
                pre_weight= [0.066,0.066,0.066,0.066,0.066,0.066,0.066,0.066,0.066,0.4]
            else:
                pre_weight= [0.088,0.088,0.088,0.088,0.088,0.088,0.088,0.088,0.088,0.2]
        
        for i in range(0,len(self.start_date_list),3):
            df_lookback = self.df.return_lookback(self.df_investment, self.start_date_list[i],lookback)
            optim = optimize_portfolio(df_lookback, asset_type)
            #경제상황
            eco_status = eco_status_arr[i//3]
            optim_weight = optim.min_variance(optim.mean_returns, optim.cov_matrix,inv_style, eco_status,pre_weight).x
            pre_weight = optim_weight
            weight_arr = np.append(weight_arr, optim_weight)
        weight_arr = weight_arr.reshape(-1,self.asset_num)
        return weight_arr
    
    def get_equal_weight(self, start_date_list, inv_style, eco_status):
        weight_equal_arr = np.array([])
        for i in range(0,len(start_date_list),3):
            equal_weight = np.ones((1,self.asset_num))/self.asset_num
            weight_equal_arr = np.append(weight_equal_arr, equal_weight)
        weight_equal_arr = weight_equal_arr.reshape(-1,self.asset_num)
        return weight_equal_arr
    
    def create_backtest_table(self, df_investment, start_date, end_date, weight_arr):
        df_rt = pd.DataFrame(columns=df_investment.columns)
        df1 = df_investment[start_date:end_date]
        money0 = weight_arr[0] * self.money
        for i in range(0, df1.shape[0]):
            money1 = (df1.iloc[i,:]/100 + 1) * money0
            if (i % 3 == 2) & (i != (df1.shape[0]-1)):
                money1 = weight_arr[(i//3+1)] * sum(money1)
            elif (i % 3 == 2) & (i == (df1.shape[0]-1)):
                pass
            money0 = money1
            df_rt = df_rt.append(pd.DataFrame([money1], columns=df_investment.columns), ignore_index=True)
        df_rt.set_index(df1.index, inplace=True)
        return df_rt
    
    def create_equal_backtest_table(self, df_investment, start_date, end_date, weight_equal_arr):
        df_rt = pd.DataFrame(columns=df_investment.columns)
        df1 = df_investment[start_date:end_date]
        money0 = weight_equal_arr[0] * self.money
        for i in range(0, df1.shape[0]):
            money1 = (df1.iloc[i,:]/100 + 1) * money0
            if (i % 3 == 2) & (i != (df1.shape[0]-1)):
                money1 = weight_equal_arr[(i//3+1)] * sum(money1)
            elif (i % 3 == 2) & (i == (df1.shape[0]-1)):
                pass
            money0 = money1
            df_rt = df_rt.append(pd.DataFrame([money1], columns=df_investment.columns), ignore_index=True)
        df_rt.set_index(df1.index, inplace=True)
        return df_rt
            
    def make_dict(self, df_rt): #결과창에서 사용할 dictionary 반환
        df_rt_pfo_ret = np.array(df_rt.sum(axis=1).pct_change())
        df_rt_pfo_ret[0] = 0
        df_rt_pfo_idx = np.array(df_rt.sum(axis=1) / df_rt.sum(axis=1)[0] * 100)
        pfo_dict = dict({'pfo_ret':df_rt_pfo_ret, 'pfo_idx':df_rt_pfo_idx})
        return pfo_dict    

class result:  #결과 그래프 클래스
    
    def __init__(self, start_date, end_date):
        tmp = PreProcess(start_date, end_date)
        start = tmp.start_date
        end = tmp.end_date
        start = pd.to_datetime(start, format = '%Y-%m-%d').strftime('%Y-%m')
        end = pd.to_datetime(end, format = '%Y-%m-%d').strftime('%Y-%m')
        df_tmp = pd.read_csv('sp_data.csv')
        df_tmp.index = df_tmp['Date']
        df_tmp = df_tmp.iloc[:,1]
        benchmark_df = df_tmp[start:end]
        self.benchmark_df = pd.DataFrame(benchmark_df)
        self.benchmark_df['n_idx']= self.benchmark_df['Close']/self.benchmark_df['Close'][0]*100
        self.benchmark_df['benchmark_ret'] = self.benchmark_df['n_idx'].pct_change()
        self.benchmark_df['benchmark_ret'][0] = 0
        self.fig_list = []
    
    def benchmark_optimizedpfo(self, pfo,pfo_equal):
        new_df = pd.DataFrame({'Benchmark_Close': self.benchmark_df['n_idx'],'Portfolio_Close':pfo['pfo_idx'],'Equal_Close':pfo_equal['pfo_idx'], 
                               'Benchmark_ret':self.benchmark_df['benchmark_ret'],'Portfolio_ret':pfo['pfo_ret'],'Equal_ret':pfo_equal['pfo_ret']})
        return new_df
    
    def drawIndex(self, df):
        title = '포트폴리오 가치 변화'
        labels = ['Optimized Portfolio', 'Benchmark','Equally-weighted Portfolio']
        colors = ['rgb(49,130,189)', 'rgb(115,115,115)','rgb(158,2,85)']
        mode_size = [12, 8,8]
        line_size = [4, 2,2]
        x_data = np.vstack((df.index.tolist(),)*3)
        y_data = np.array([
            df['Portfolio_Close'],
            df['Benchmark_Close'],
            df['Equal_Close']])
        fig = go.Figure()
        for i in range(0, 3):
            fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
                name=labels[i],
                line=dict(color=colors[i], width=line_size[i]),
                connectgaps=True))
        fig.update_layout(
            xaxis=dict(
                showline=True,
                showgrid=False,
                showticklabels=True,
                linecolor='rgb(204, 204, 204)',
                linewidth=2,
                ticks='outside',
                tickfont=dict(
                    family='Arial',
                    size=12,
                    color='rgb(82, 82, 82)')),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
            ),
            autosize=True,
            margin=dict(
                autoexpand=True,
                l=50,
                r=20,
                t=200,
            ),
            showlegend=True,
            legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01),
            plot_bgcolor='white')
        annotations = []
        # Title
        annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                      xanchor='left', yanchor='bottom',
                                      text='포트폴리오 가치 변화',
                                      font=dict(family='Arial',
                                                size=30,
                                                color='rgb(37,37,37)'),
                                      showarrow=False))
        # Source
        annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                                      xanchor='center', yanchor='top',
                                      text='Investment Period',
                                      font=dict(family='Arial',
                                                size=12,
                                                color='rgb(150,150,150)'),
                                      showarrow=False))
        fig.update_layout(annotations=annotations)
        self.fig_list.append(fig)
        
    def drawReturn(self,df):
        title = 'Return Graph'
        labels = ['Optimized Portfolio', 'Benchmark','Equally-weighted Portfolio']
        colors = ['rgb(255,165,0)', 'rgb(115,115,115)','rgb(158,2,85)']
        mode_size = [12, 8,8]
        line_size = [4, 2,2]
        x_data = np.vstack((df.index.tolist(),)*3)
        y_data = np.array([
            df['Portfolio_ret'],df['Benchmark_ret'],df['Equal_ret']])
        fig = go.Figure()
        for i in range(0, 3):
            fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',name=labels[i],
                line=dict(color=colors[i], width=line_size[i]),connectgaps=True))
        fig.update_layout(
            xaxis=dict(
                showline=True,
                showgrid=False,
                showticklabels=True,
                linecolor='rgb(204, 204, 204)',
                linewidth=2,
                ticks='outside',
                tickfont=dict(
                    family='Arial',
                    size=12,
                    color='rgb(82, 82, 82)')),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True),
            autosize=True,
            margin=dict(
                autoexpand=True,
                l=50,
                r=20,
                t=200),
            showlegend=True,
            legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01),
            plot_bgcolor='white')
        annotations = []
        # Title
        annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                      xanchor='left', yanchor='bottom',
                                      text='Return Graph',
                                      font=dict(family='Arial',
                                                size=30,
                                                color='rgb(37,37,37)'),
                                      showarrow=False))
        # Source
        annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                                      xanchor='center', yanchor='top',
                                      text='Investment Period',
                                      font=dict(family='Arial',
                                                size=12,
                                                color='rgb(150,150,150)'),
                                      showarrow=False))
        fig.update_layout(annotations=annotations)
        self.fig_list.append(fig)

    def drawDoughnut(self, weight):
        g_count = len(weight) 
        labels = BackTest.df_investment.columns ######################################여기
        date_list=[]
        for i in range(Pprocess.calculate_period(Pprocess.start_date, Pprocess.end_date)//12+1):
            date_list.append(Pprocess.start_date + relativedelta(years= i ))
        date_list.append(Pprocess.end_date)
        for i in range(g_count//4 if g_count%4==0 else g_count//4+1):
            specs = [[{'type':'domain'}for i in range(4)]]
            fig = make_subplots(rows=1, cols=4, specs=specs)
            for j in range(4):
                try:
                    fig.add_trace(go.Pie(labels=labels, values=weight[i*4+j], name='1'),1, j+1)
                except:
                    break
        # Use `hole` to create a donut-like pie chart
            fig.update_traces(hole=.4, hoverinfo="label+percent+name")
            fig.update_layout(
                title_text="Investment period "+datetime.datetime.strftime(date_list[i],'%Y%m')+"-"+datetime.datetime.strftime(date_list[i+1],'%Y%m'),
                margin=dict(t=200, b=0, l=0, r=0),
                autosize=False,
                width=1500,
                height=450)
            self.fig_list.append(fig)
        
    def drawMDD(self,df):
        pfo_highest_past = []
        bcm_highest_past = []
        eql_highest_past = []
        for i in range(1,len(df)+1):
            pfo_highest_past.append(max(df['Portfolio_Close'].values[0:i]))
            bcm_highest_past.append(max(df['Benchmark_Close'].values[0:i]))
            eql_highest_past.append(max(df['Equal_Close'].values[0:i]))
        pfo_drawdown = df['Portfolio_Close'].values/pfo_highest_past - 1
        bcm_drawdown = df['Benchmark_Close'].values/bcm_highest_past - 1
        eql_drawdown = df['Equal_Close'].values/eql_highest_past - 1
        pfo_MDD = min(pfo_drawdown)
        bcm_MDD = min(bcm_drawdown)
        eql_MDD = min(eql_drawdown)
        pfo_MDDindex= pfo_drawdown==pfo_MDD
        pfo_MDDindex=[i for i, x in enumerate(pfo_MDDindex) if x]
        bcm_MDDindex= bcm_drawdown==bcm_MDD
        bcm_MDDindex=[i for i, x in enumerate(bcm_MDDindex) if x]
        eql_MDDindex= eql_drawdown==eql_MDD
        eql_MDDindex=[i for i, x in enumerate(eql_MDDindex) if x]

        title = 'DrawDown'
        labels = ['Optimized Portfolio', 'Benchmark','Equally-weighted Portfolio']
        colors = ['rgb(255,0,0)', 'rgb(115,115,115)','rgb(158,2,85)']
        mode_size = [12, 8, 8]
        line_size = [4, 2, 2]
        x_data = np.vstack((df.index.tolist(),)*3)
        y_data = np.array([
            pfo_drawdown,
            bcm_drawdown,
            eql_drawdown])
        fig = go.Figure()
        for i in range(0, 3):
            fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
                name=labels[i],
                line=dict(color=colors[i], width=line_size[i]),
                connectgaps=True))

        fig.add_trace(go.Scatter(x=df.index[pfo_MDDindex], y=pfo_drawdown[pfo_MDDindex], mode='markers',name='Portfolio_MDD', marker_size=20,
                                marker_color='#ff7f50'))
        fig.add_trace(go.Scatter(x=df.index[bcm_MDDindex], y=bcm_drawdown[bcm_MDDindex], mode='markers',name='Benchmark_MDD', marker_size=20,
                                marker_color='#4ca3dd'))
        fig.add_trace(go.Scatter(x=df.index[eql_MDDindex], y=eql_drawdown[eql_MDDindex], mode='markers',name='Equal-weighted_MDD', marker_size=20,
                                marker_color='#f3bfc3'))
        fig.update_layout(
            xaxis=dict(
                showline=True,
                #showgrid=False,
                showticklabels=True,
                linecolor='rgb(204, 204, 204)',
                linewidth=2,
                ticks='outside',
                tickfont=dict(
                    family='Arial',
                    size=12,
                    color='rgb(82, 82, 82)')),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True),
            autosize=True,
            margin=dict(
                autoexpand=True,
                l=50,
                r=20,
                t=200),
            showlegend=True,
            legend=dict(
            yanchor="top",
            y=1,
            xanchor="left",
            x=1),
            plot_bgcolor='white')
        annotations = []
        # Title
        annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                      xanchor='left', yanchor='bottom',
                                      text=title,
                                      font=dict(family='Arial',
                                                size=30,
                                                color='rgb(37,37,37)'),
                                      showarrow=False))
        # Source
        annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                                      xanchor='center', yanchor='top',
                                      text='Investment Period',
                                      font=dict(family='Arial',
                                                size=12,
                                                color='rgb(150,150,150)'),
                                      showarrow=False))
        fig.update_layout(annotations=annotations)
        self.fig_list.append(fig)
    def VaRgraph(self, df):
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=df['Portfolio_ret'],name='Portfolio'))
        fig.add_trace(go.Histogram(x=df['Benchmark_ret'],name='Benchmark'))
        fig.update_layout(
            barmode='overlay',
            #title_text='Return histogram', # title of plot
            xaxis_title_text='Returns', # xaxis label
            yaxis_title_text='Count', # yaxis label
            bargap=0.2, # gap between bars of adjacent location coordinates
            bargroupgap=0.1, # gap between bars of the same location coordinates
            autosize=True,
            margin=dict(
                autoexpand=True,
                l=50,
                r=20,
                t=200))
        title = 'Return histogram'
        fig.update_traces(opacity=0.75)
        annotations = []
        annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                              xanchor='left', yanchor='bottom',
                                              text=title,
                                              font=dict(family='Arial',
                                                        size=30,
                                                        color='rgb(37,37,37)'),
                                              showarrow=False))
        # Source
        annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                                      xanchor='center', yanchor='top',
                                      text='-',
                                      font=dict(family='Arial',
                                                size=12,
                                                color='rgb(150,150,150)'),
                                      showarrow=False))
        fig.update_layout(annotations=annotations)
        self.fig_list.append(fig)
        
    def resultTable(self,df):
        Bmean_ret = df['Benchmark_ret'].mean()
        Pmean_ret = df['Portfolio_ret'].mean()
        Emean_ret = df['Equal_ret'].mean()
        
        Bstd = df['Benchmark_ret'].std()
        Pstd = df['Portfolio_ret'].std()
        Estd = df['Equal_ret'].std()
        
        risk_free_rate = 0.0025
        
        Bsharpe = (Bmean_ret-risk_free_rate)/Bstd 
        Psharpe = (Pmean_ret-risk_free_rate)/Pstd 
        Esharpe = (Emean_ret-risk_free_rate)/Estd
        
        PTErr = (df['Portfolio_ret']-df['Benchmark_ret']).std()
        ETErr = (df['Equal_ret']-df['Benchmark_ret']).std()
        PIratio = (df['Portfolio_ret']-df['Benchmark_ret']).mean()/PTErr
        EIratio = (df['Equal_ret']-df['Benchmark_ret']).mean()/ETErr
        MAR = 0
        df['ret-MAR']=df['Portfolio_ret']-MAR
        df['Equal_ret-MAR']=df['Equal_ret']-MAR
        df['Benchmark_ret-MAR']=df['Benchmark_ret']-MAR
        
        P_downside_ret = []
        for i in range(len(df)):
            P_downside_ret.append(min(0,df['ret-MAR'][i])**2)
        PDSstd = np.sqrt(sum(P_downside_ret)/(len(P_downside_ret)-1))
        
        E_downside_ret = []
        for i in range(len(df)):
            E_downside_ret.append(min(0,df['Equal_ret-MAR'][i])**2)
        EDSstd = np.sqrt(sum(E_downside_ret)/(len(E_downside_ret)-1))
        
        B_downside_ret = []
        for i in range(len(df)):
            B_downside_ret.append(min(0,df['Benchmark_ret-MAR'][i])**2)
        BDSstd = np.sqrt(sum(B_downside_ret)/(len(B_downside_ret)-1))
        
        PSratio = Pmean_ret/PDSstd
        ESratio = Emean_ret/EDSstd
        BSratio = Bmean_ret/BDSstd
        
        sort_df=df.sort_values('Portfolio_ret',ascending=True)
        pfo_VaR_95 = sort_df['Portfolio_ret'].quantile(0.05)
        pfo_CVaR_95 = sort_df[sort_df['Portfolio_ret']<pfo_VaR_95]['Portfolio_ret'].mean()
        
        sort_df=df.sort_values('Benchmark_ret',ascending=True)
        bcm_VaR_95 = sort_df['Benchmark_ret'].quantile(0.05)
        bcm_CVaR_95 = sort_df[sort_df['Benchmark_ret']<bcm_VaR_95]['Benchmark_ret'].mean()
        
        sort_df=df.sort_values('Equal_ret',ascending=True)
        equal_VaR_95 = sort_df['Equal_ret'].quantile(0.05)
        equal_CVaR_95= sort_df[sort_df['Equal_ret']<equal_VaR_95]['Equal_ret'].mean()
        
        headerColor = 'grey'
        fig = go.Figure(data=[go.Table(
          header=dict(
            values=['<b></b>','<b>Benchmark</b>','<b>Optimized Portfolio</b>','<b>Equally weighted Portfolio</b>'],
            line_color='darkslategray',
            fill_color=headerColor,
            align=['left','center'],
            font=dict(color='white', size=15),
            height=50,
          ),
          cells=dict(
            values=[
              ['Annual return','volatility','Sharpe ratio','Sortino ratio','Tracking Error','Information ratio','VaR 95%', 'CVaR 95%'],
              [np.round(Bmean_ret*12,3), np.round(Bstd,3),
               np.round(Bsharpe,3),np.round(BSratio,3),'-','-', np.round(bcm_VaR_95,3),np.round(bcm_CVaR_95,3)],
                [np.round(Pmean_ret*12,3), np.round(Pstd,3),
               np.round(Psharpe,3),np.round(PSratio,3),np.round(PTErr,3),np.round(PIratio,3), np.round(pfo_VaR_95,3),np.round(pfo_CVaR_95,3)],
                [np.round(Emean_ret*12,3), np.round(Estd,3),
                 np.round(Esharpe,3),np.round(ESratio,3),np.round(ETErr,3),np.round(EIratio,3), np.round(equal_VaR_95,3),np.round(equal_CVaR_95,3)]],
            line_color='darkslategray',
            # 2-D list of colors for alternating rows
            fill_color = 'white',
            align = ['left', 'center'],
            font = dict(color = 'darkslategray', size = 20),
            height = 25
            ))
        ])
        fig.update_layout(
                    #title_text='성과 비교', # title of plot
                    autosize=True,
                    margin=dict(
                        autoexpand=True,
                        l=50,
                        r=20,
                        t=200,
                    ),
            
            
                )
        title= 'Performance Evaluation'
        annotations = []
        annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                              xanchor='left', yanchor='bottom',
                                              text=title,
                                              font=dict(family='Arial',
                                                        size=30,
                                                        color='rgb(37,37,37)'),
                                              showarrow=False))

        fig.update_layout(annotations=annotations)
        self.fig_list.append(fig)


In [4]:
if __name__ =='__main__':
    print('==============================================================')
    asset_type = input("투자자산은 무엇인가요?    *예)1 : fama5, 2 :fama10 \n")
    
    print('==============================================================')
    money = int(input("투자금액은 얼마인가요? \n"))
    print("==============================================================")
    
    print("A에 아주 가깝다 : 4")
    print("A에 가까운 편이다 : 3")
    print("B에 가까운 편이다 : 2")
    print("B에 아주 가깝다 : 1")
    print("해당 번호를 입력해주세요")
    print("==============================================================")
    
    
    num1 = int(input("A.주식, 채권 등 직접 투자를 원한다.   B. 펀드와 같은 간접투자를 선호한다.\n"))
    num2 = int(input("A.단기적으로 투자한다.   B.장기적으로 투자한다.\n"))
    num3 = int(input("A.좋은 투자기회가 있다면 빚을 내서라도 투자한다.   B.좋은 투자기회가 있더라도, 여유자금 내에서 투자한다.\n"))
    num4 = int(input("A.한 가지 상품에 집중적으로 투자한다.   B.다양한 상품에 분산하여 투자한다.\n"))
    num5 = int(input("A.투자에서는 수익성이 우선이다.   B.투자의 안전성이 우선이다.\n"))
    
    tot = num1+num2+num3+num4+num5
    #1은 안정형, 2은 중립, 3은 공격
    if (tot<=10):
        inv_style= 1
        text= '안정형'
    elif (tot>15):
        inv_style=3
        text = '공격형'
    else: 
        inv_style=2 
        text = '중립형'
        
    print('==============================================================')
    print("당신의 투자 성향은", text, "입니다.")
    print('==============================================================')
    start_date = input("투자 시작 날짜를 입력해주세요.   *예)202001 \n")
    end_date = input("투자 마지막 날짜를 입력해주세요.    *예)202011 \n")
    lookback = input("lookback 기간을 입력해주세요.    *예)연단위 : 1,2,3... \n")
    #rebalancing = input("rebalancing 기간을 입력해주세요.    *예)월단위 : 1,2,...12 \n")
    print('==============================================================')
    
    econ_train1 = predict.get_data_econ_1_train()  # 모델1 트레이닝관련자료 (1960-1999)
    econ_train2 = predict.get_data_econ_2_train() # 모델2 트레이닝관련자료 (1960-1999)
    
    econ_test1 = predict.get_data_econ_1_test() # 모델1 예측해야하는 자료 (2000-2020)
    econ_test2 = predict.get_data_econ_2_test() # 모델2 예측해야하느 자료 (2000-2020)
    

    print('==============================================================')
    
    print('lookback 기간은 %s년, rebalancing period는 3개월입니다.' %lookback)
    
    print("===============================================================")
    Sr = input("결과를 보시겠습니까? (y or n)\n")
    Pprocess = PreProcess(start_date, end_date)
    eco_status_arr= predict.eco_status_predict(Pprocess.start_date,Pprocess.end_date)
    
    
    if(Sr == 'y'):
        
        BackTest = Backtest(start_date, end_date, money, asset_type)
        weight = BackTest.get_rebalance_weight_optimize_portfolio(BackTest.start_date_list,inv_style, eco_status_arr,lookback)
        table = BackTest.create_backtest_table(BackTest.df_investment, BackTest.start_date, BackTest.end_date, weight)
        weight_equal = BackTest.get_equal_weight(BackTest.start_date_list,inv_style, eco_status_arr)
        table_equal = BackTest.create_equal_backtest_table(BackTest.df_investment, BackTest.start_date, BackTest.end_date, weight_equal)
        pfo = BackTest.make_dict(table)
        pfo_equal=BackTest.make_dict(table_equal)
        mod = result(start_date,end_date)
        df = mod.benchmark_optimizedpfo(pfo,pfo_equal)
        mod.drawIndex(df)
        mod.drawReturn(df)
        
        investment_period= Pprocess.calculate_period(Pprocess.start_date, Pprocess.end_date)
        mod.drawDoughnut(weight)
        mod.drawMDD(df)
        mod.VaRgraph(df)
        mod.resultTable(df)
        
        filename=f"{os.path.join('', 'Result.html')}"
        dashboard = open(filename, 'w')
        dashboard.write("<html><head></head><body>" + "\n")
        dashboard.write("<h2 style='text-align:center;'>당신의 투자성향은 "+text+" 입니다</h2>\n")
        dashboard.write("<h3 style='text-align:center;'>Lookback 기간 : "+lookback+" 년</h3>\n")
        dashboard.write("<h3 style='text-align:center;'>Portfolio rebalancing : 3 개월</h3>\n")
        dashboard.write("<h3 style='text-align:center;'>Total investment period : "+str(investment_period)+"개월</h3>\n")

        include_plotlyjs = True

        for fig in mod.fig_list:
            inner_html = fig.to_html(include_plotlyjs = include_plotlyjs).split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
            include_plotlyjs = False
            #display(fig)
        dashboard.write("</body></html>" + "\n")
        print("최적화 완료")
    
        

투자자산은 무엇인가요?    *예)1 : fama5, 2 :fama10 
1
투자금액은 얼마인가요? 
1000
A에 아주 가깝다 : 4
A에 가까운 편이다 : 3
B에 가까운 편이다 : 2
B에 아주 가깝다 : 1
해당 번호를 입력해주세요
A.주식, 채권 등 직접 투자를 원한다.   B. 펀드와 같은 간접투자를 선호한다.
1
A.단기적으로 투자한다.   B.장기적으로 투자한다.
2
A.좋은 투자기회가 있다면 빚을 내서라도 투자한다.   B.좋은 투자기회가 있더라도, 여유자금 내에서 투자한다.
3
A.한 가지 상품에 집중적으로 투자한다.   B.다양한 상품에 분산하여 투자한다.
4
A.투자에서는 수익성이 우선이다.   B.투자의 안전성이 우선이다.
1
당신의 투자 성향은 중립형 입니다.
투자 시작 날짜를 입력해주세요.   *예)202001 
200101
투자 마지막 날짜를 입력해주세요.    *예)202011 
200501
lookback 기간을 입력해주세요.    *예)연단위 : 1,2,3... 
1
lookback 기간은 1년, rebalancing period는 3개월입니다.
결과를 보시겠습니까? (y or n)
y


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.20.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


최적화 완료
